**AI & Machine Learning (KAN-CINTO4003U) - Copenhagen Business School | Spring 2025**

***


<p align="center">
<img src="media/tools_header.webp" alt="LLM" width="800"/> <br>
Image from <a href="https://medium.com/@lorevanoudenhove">Lore Van Oudenhove<a/>'s "<i><a href="https://medium.com/@lorevanoudenhove/how-to-build-ai-agents-with-langgraph-a-step-by-step-guide-5d84d9c7e832">How to Build AI Agents with LangGraph</a></i>". <br>Copyright © 2025. All rights reserved.
</p>

***
Sources: <br>
- [How to Build AI Agents with LangGraph (Lore Van Oudenhove via Medium)](https://medium.com/@lorevanoudenhove/how-to-build-ai-agents-with-langgraph-a-step-by-step-guide-5d84d9c7e832)


# Tool-using LLM agents

In [`guides/router_agents.ipynb`](router_agents_guide.ipynb), we built a simple `router` agent. This introduces us to the concept of using LLMs as reasoning engines that influence which steps to take to complete a task. In this notebook, we will build on this concept and create a more complex agent that can use tools to complete tasks.

As last time, an LLM Agent will

1. Receive a sensory input (e.g. a text)
2. Decide what to do with it
3. Take an action (e.g. generate a summary)

However, for tool using agents, these steps are taken in a loop. The agent will receive an input, decide what to do with it, take an action, and then receive the output of that action as input for the next step. This process continues until the agent decides to stop.

As in the last notebook, we will be using [LangGraph](https://langchain-ai.github.io/langgraph/) to build our agent. The prior notebook introduces important concepts like `StateGraph`, `State`, `Node`, and `Edge`. Please make sure to start there.






### Setup

In [ ]:
# langgraph/langchain libraries
from langgraph.prebuilt import create_react_agent
from langchain_ibm import ChatWatsonx
from langchain_ibm import WatsonxEmbeddings
from langchain_chroma import Chroma
from langchain_core.tools import tool
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools import DuckDuckGoSearchResults

# misc libraries
import requests
from decouple import config

Let's start by loading our WatsonX.ai credentials again

In [ ]:
WX_API_KEY = config("WX_API_KEY")
WX_PROJECT_ID = config("WX_PROJECT_ID")
WX_API_URL = "https://us-south.ml.cloud.ibm.com"

We can use the `ChatWatsonx` class from `langchain_ibm` - a LangChain wrapper for WatsonX.ai - to create a chat model. . We can use it to create a chat model that can be used in LangGraph.

In [ ]:
chat_model = ChatWatsonx(
    url=WX_API_URL,
    apikey=WX_API_KEY,
    project_id=WX_PROJECT_ID,
    model_id="mistralai/mistral-large",
)

# Using a prebuilt agent architecture

LangGraph offers a prebuilt agent architecture that we can use simply by calling `create_react_agent`. This function takes as inputs (among other more advanced options)

- model: the LLM model to use (In our case `ChatWatsonx`)
- tools: the tools to use as a list

So, first we need to define some tools. We can easily do so by using the `@tool` decoractor from `langgraph` as shown below.

In [ ]:
@tool
def add(a: float, b: float) -> float:
    """Add a and b."""
    return a + b

@tool
def subtract(a: float, b: float) -> float:
    """Subtract a and b."""
    return a - b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply a and b."""
    return a * b

@tool
def divide(a: float, b: float) -> float:
    """Divide a and b."""
    return a / b

# and to make it a little more interesting, let's add a currency converter
@tool
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date.

    Uses the Frankfurter API (https://api.frankfurter.app/) to obtain
    exchange rate data.

    Args:
        currency_from: The base currency (3-letter currency code).
            Defaults to "USD" (US Dollar).
        currency_to: The target currency (3-letter currency code).
            Defaults to "EUR" (Euro).
        currency_date: The date for which to retrieve the exchange rate.
            Defaults to "latest" for the most recent exchange rate data.
            Can be specified in YYYY-MM-DD format for historical rates.

    Returns:
        dict: A dictionary containing the exchange rate information.
            Example: {"amount": 1.0, "base": "USD", "date": "2023-11-24",
                "rates": {"EUR": 0.95534}}
    """
    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()


tools = [add, subtract, multiply, divide, get_exchange_rate]

Here is our final list of tools:

In [ ]:
tools

Now, with our model and tools defined, we can create our agent.

In [ ]:
graph = create_react_agent(chat_model, tools=tools, debug=True)

graph

Let's try it out

In [ ]:
inputs = {
    "messages": [
        (
            "user", "I have 833 USD and my sister has 38 USD. How much do we have total in euros?"
        )
    ]
}

# we set the debug flag to True to print tasks and their results
response = graph.invoke(inputs, debug=True)

Our response will be a set of messages

In [ ]:
response

And the last AIMessage will be the final response from the model

In [ ]:
response["messages"][-1].content

***

### Adding a RAG tool

Let's see how we can add a RAG tool to our agent. We can use the `create_retriever_tool` from `langgraph` to create a retriever tool. This tool will allow the agent to retrieve information from a knowledge base and use it to answer questions.

If you have already run everything in the [`rag_guide.ipynb`](rag_guide.ipynb) notebook, you will already have a local vector DB (by default called `my_vector_db`) with some documents in it. If you haven't, please run the notebook first to create the vector DB. You can also use any other vector DB you have access to.

Since we will also have to embed incoming queries, we first need to define our embedding model here as well.

In [ ]:
embed_params = {}

watsonx_embedding = WatsonxEmbeddings(
    model_id="ibm/granite-embedding-278m-multilingual",
    url=WX_API_URL,
    project_id=WX_PROJECT_ID,
    apikey=WX_API_KEY,
    params=embed_params,
)

Then, let's initialize our existing vector database.

In [ ]:
vectordb = Chroma(collection_name="my_collection", persist_directory="my_vector_db", embedding_function=watsonx_embedding)

vectordb.get(limit=1)

In [ ]:
retriever_tool = create_retriever_tool(
    vectordb.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 3,
        }
    ),
    "retrieve_company_information",
    "Search and return information about MadeUpCompany",
)

tools_for_agent = [get_exchange_rate, retriever_tool]

In [ ]:
# Use the vectorstore as a retriever
retriever = vectordb.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 3,
    }
)

retrieved_documents = retriever.invoke("Do you have a 30-day money-back guarantee?")

In [ ]:
retrieved_documents

In [ ]:
graph = create_react_agent(chat_model, tools=tools_for_agent, debug=True)

In [ ]:
response = graph.invoke(
    
    {
        "messages": [
            (
                "user", "What is CloudMate from MadeUpCompany? Keep it short and simple."
            )
        ],
    },
    debug=True,
    
)

In [ ]:
response["messages"][-1].content

In [ ]:
response = graph.invoke(
    
    {
        "messages": [
            (
                "user", "What is the price of CloudMate Professional? Find the price first and then convert it to euros."
            )
        ],
    },
    debug=True,
    
)

In [ ]:
print(response["messages"][-1].content)

### Adding a search tool

In [ ]:
search_tool = DuckDuckGoSearchResults()

In [ ]:
tools_for_agent = [get_exchange_rate, retriever_tool, search_tool]

graph = create_react_agent(chat_model, tools=tools_for_agent, debug=True)

In [ ]:
tools_for_agent

In [ ]:
response = graph.invoke(
    {
        "messages": [
            (
                "user", "Compare the price of CloudMate Professional from MadeUpCompany with the price of OneDrive from Microsoft. Find the price first and then convert it to euros."
            )
        ],
    },
    debug=True,
    
)

In [ ]:
response["messages"][-1].content

# Next steps

We can add a many different tools to our react agent. You can find a list of all available tools in the [LangGraph documentation on tools](https://python.langchain.com/docs/integrations/tools/). Some of the most common tools are:

- `Calculator`: A calculator tool that can perform basic arithmetic operations.
- `Python`: A Python REPL that can execute Python code. (Be careful with this one, as it can execute any code you give it. This can be dangerous if you are not careful.)
- `Office365 Toolkit`: A toolkit that can be used to interact with Office365 applications like Excel, Word, and PowerPoint.
- `SQLDatabase Toolkit` : A toolkit that can be used to interact with SQL databases.
- `ArXiv` : A tool that can be used to search for papers on ArXiv.


One of the disadvantes of using the `create_react_agent` function is that it does not allow us to customize the agent's behavior much. If we want to customize the agent's behavior, we need to create our own agent using the `StateGraph` class.